In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import math
import serial
from threading import Thread
import time
import re

import pylab as pl
from IPython import display
import matplotlib.pyplot as plt

import cv2
import numpy as np
from utils.depth_manager import *
from utils.projections import *
from tqdm import tqdm

In [3]:
%matplotlib qt

In [4]:
# connect to the Serial
SERIAL_CONNECTION = serial.Serial('COM3', 9600)
time.sleep(2)

# Utils

In [5]:
def parse_serial(serial_msg):
    """
    Function to parse serial data to extract float values
    format 'x:19.34 y:23.01 z:-33.83' to x, y, z float values

    Args:
        - (str) string with format 'x:19.34 y:23.01 z:-33.83'
    Return:
        - (list) x, y, z float values
    """
    xyz_list = re.findall('[-+]?[0-9]*\.?[0-9]*', serial_msg)
    return [float(i) for i in filter(lambda item: item, xyz_list)]

In [6]:
def UpdateOrientation(ser):
    """
    Function to integration the x data from the Gyroscope
    and update the global variable x_orientation with the new value

    Args:
        - (serial.Serial) serial to get the gyroscope data
    """
    global X_ORIENTATION

    while True:
        serial_msg_bytes = ser.readline()
        serial_msg = serial_msg_bytes.decode()
        dx, dy, dz = parse_serial(serial_msg)
        
        # The gyroscope values are in degrees-per-second
        # divide each value by the number of samples per second
        dx_normalized = dx / GYROSCOPE_SAMPLE_RATE;

        # remove noise
        if(abs(dx_normalized) > 0.004):
            # update orientation
            X_ORIENTATION = X_ORIENTATION - dx_normalized*1.25
            X_ORIENTATION = X_ORIENTATION%360

# Update orientation in real time

In [28]:
# initialize x orientation
X_ORIENTATION = 0

# 119 got from Arduino with IMU.gyroscopeSampleRate();
GYROSCOPE_SAMPLE_RATE= 119

In [8]:
# run the thread to update the x orientation in real time
Thread(target=UpdateOrientation, args=(SERIAL_CONNECTION,)).start()

# Get depth image and do projection

In [9]:
INTERPRETER = get_tflite_interpreter(
    "https://tfhub.dev/intel/lite-model/midas/v2_1_small/1/lite/1?lite-format=tflite",
    "model/midas_v2_1_small.tflite")

# Calcul projection in real time

In [29]:
# Get input image
IMG = cv2.imread('images/dog.jpg')
RGB_IMG = cv2.cvtColor(IMG, cv2.COLOR_BGR2RGB)

FIG = plt.figure()

POINTS_IN_NED = np.array([])
DEPTH_VALUES = []

PROJECT_DEPTH = True

DEGREES_INTERVAL = 60

ORIENTATIONS_DONE = []
ORIENTATIONS_TODO = [orientation for orientation
                     in range(0, 359, DEGREES_INTERVAL)]

try:
    while True:
        POINTS_IN_NED, DEPTH_VALUES = \
        plot_env(FIG, X_ORIENTATION, POINTS_IN_NED, DEPTH_VALUES,
                 RGB_IMG, INTERPRETER, PROJECT_DEPTH, ORIENTATIONS_DONE,
                 ORIENTATIONS_TODO, pourcentage_to_project=2)
        if len(ORIENTATIONS_TODO) == 0:
            break
except KeyboardInterrupt:
    pass    

plt.show()